In [2]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# 1. 데이터 전처리
# 펭귄 데이터셋에는 결측치가 있으므로, .dropna()로 제거합니다.
penguins = sns.load_dataset("penguins")
penguins_processed = penguins.dropna()

# 2. 피처(X)와 타겟(y) 분리
# 'species' 열을 예측 대상(y)으로, 나머지를 특성(X)으로 사용합니다.
X = penguins_processed.drop('species', axis=1)
y = penguins_processed['species']

# 3. 인코딩
# [피처 인코딩]
# 머신러닝 모델은 문자열을 이해하지 못하므로, 범주형 피처('island', 'sex')를 숫자로 변환합니다.
# pd.get_dummies를 사용해 원-핫 인코딩을 수행합니다.
X = pd.get_dummies(X, columns=['island', 'sex'], drop_first=True)

# [타겟 인코딩]
# 타겟 변수(species)도 문자열이므로 LabelEncoder를 사용해 0, 1, 2와 같은 숫자로 변환합니다.
le = LabelEncoder()
y = le.fit_transform(y)

# 4. 데이터 분할
# 데이터를 훈련용(train)과 테스트용(test)으로 분리합니다. (테스트 비율 30%)
# stratify=y : 훈련/테스트셋의 타겟 클래스 비율을 원본 데이터와 동일하게 유지합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 5. 피처 스케일링
# KNN과 같은 거리 기반 알고리즘은 피처의 스케일에 민감합니다.
# StandardScaler를 사용해 모든 피처의 스케일을 표준화합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6. 모델 정의
# 요청한 세 가지 모델을 정의합니다.
models = {
    "Naive Bayes": GaussianNB(),
    "KNN (k=3)": KNeighborsClassifier(n_neighbors=3),
    "Decision Tree (depth=3)": DecisionTreeClassifier(max_depth=3, random_state=42)
}

# 7. 모델 학습 및 평가
results = {}

print("--- 모델 학습 및 평가 시작 ---")
for name, model in models.items():
    # 모델 학습 (스케일링된 데이터 사용)
    model.fit(X_train_scaled, y_train)
    
    # 예측
    y_pred = model.predict(X_test_scaled)
    # ROC AUC 계산을 위한 확률 예측
    y_pred_proba = model.predict_proba(X_test_scaled)
    
    # 평가지표 계산
    accuracy = accuracy_score(y_test, y_pred)
    # 펭귄 품종은 3개(다중 클래스)이므로, 'average' 옵션이 필요합니다.
    # 'weighted': 클래스별 샘플 수로 가중 평균을 계산하여 클래스 불균형을 고려합니다.
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    # 다중 클래스 ROC AUC는 'multi_class'와 'average' 옵션이 필요합니다.
    # 'ovr': One-vs-Rest 방식
    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
    
    # 결과 저장
    results[name] = {
        "Accuracy": accuracy,
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall,
        "ROC AUC Score": roc_auc
    }

print("\n--- 최종 평가 결과 ---")
# 8. 결과 출력
for name, metrics in results.items():
    print(f"\n## {name} 모델")
    print("-" * (len(name) + 6))
    print(f"정확도 (Accuracy):  {metrics['Accuracy']:.4f}")
    print(f"F1 Score:        {metrics['F1 Score']:.4f}")
    print(f"Precision Score: {metrics['Precision']:.4f}")
    print(f"Recall Score:    {metrics['Recall']:.4f}")
    print(f"ROC AUC Score:   {metrics['ROC AUC Score']:.4f}")

--- 모델 학습 및 평가 시작 ---

--- 최종 평가 결과 ---

## Naive Bayes 모델
-----------------
정확도 (Accuracy):  0.7800
F1 Score:        0.7685
Precision Score: 0.8597
Recall Score:    0.7800
ROC AUC Score:   0.9994

## KNN (k=3) 모델
---------------
정확도 (Accuracy):  0.9900
F1 Score:        0.9901
Precision Score: 0.9905
Recall Score:    0.9900
ROC AUC Score:   1.0000

## Decision Tree (depth=3) 모델
-----------------------------
정확도 (Accuracy):  0.9800
F1 Score:        0.9799
Precision Score: 0.9809
Recall Score:    0.9800
ROC AUC Score:   0.9814
